In [1]:
import psycopg2
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
pd.options.display.float_format = '{:.0f}'.format

In [2]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

conn.set_session (autocommit = True)

In [3]:
users_df = pd.read_sql("select * from users", conn)
users_analytics= users_df.groupby(['gender','level'], sort = True).agg({'user_id':'count'})
users_analytics.rename(columns = {'user_id':'users_count'}, inplace = True)
print('\nusers analytics :\n')
users_analytics

songs_df = pd.read_sql("select a.*,b.* from artists a full outer join songs b \
on a.artist_id = b.artist_id", conn)
songs_analytics = songs_df.groupby(['name'], sort = True).agg({'song_id': 'count'})
songs_analytics.rename(columns = {'song_id':'songs_count'}, inplace = True)
print('\nsongs count per artist :\n')
print(songs_analytics.to_string())


songplay_df = pd.read_sql("select a.songplay_id,a.level,a.session_id, \
d.first_name user_first_name, d.last_name user_last_name, \
c.name artist_name, b.title song_name, b.duration,  \
'hour : ' || e.hour || '- day :' || e.day  || '- week :' || e.week  || '- month : ' || e.month  || '- year :' || e.year as time \
from songplay a inner join songs b \
on a.song_id = b.song_id \
inner join artists c \
on a.artist_id = c.artist_id \
inner join users d \
on a.user_id = d.user_id \
inner join time e \
on a.start_time = e.start_time", conn)

print('\nsongplay records matched with dimention tables :\n')
songplay_df

listening_time_df = pd.read_sql("select a.songplay_id,a.level,a.session_id, \
d.first_name user_first_name, d.last_name user_last_name, \
e.hour , e.day  ,e.week  , e.month ,e.year ,e.weekday \
from songplay a left join  \
users d \
on a.user_id = d.user_id \
left join time e \
on a.start_time = e.start_time", conn)

listening_time_analytics = listening_time_df.groupby(['weekday','hour'], sort = True).agg({'songplay_id': 'count'})
listening_time_analytics.rename(columns = {'songplay_id':'songplays_count'}, inplace = True)
print('\nsongplays count per time items :\n')
print(listening_time_analytics.to_string())






users analytics :



users_count
gender level             
F      free            44
       paid            11
M      free            37
       paid             4


songs count per artist :

                                                                                                songs_count
name                                                                                                       
40 Grit                                                                                                   1
Adam Ant                                                                                                  1
Alice Stuart                                                                                              1
Andy Andy                                                                                                 1
Backstreet Boys                                                                                           1
Billie Jo Spears                                                                                          1
Bitter End                                                                                                1
B

,songplay_id,level,session_id,user_first_name,user_last_name,artist_name,song_name,duration,time
0,4627,paid,818,Lily,Lily,Elena,Setanta matins,270,hour : 21- day :21- week :47- month : 11- year...



songplays count per time items :

              songplays_count
weekday hour                 
0       0                   6
        1                  11
        2                  36
        3                  22
        4                  28
        5                  23
        6                  31
        7                  30
        8                  46
        9                  50
        10                 69
        11                 52
        12                 42
        13                 49
        14                 72
        15                 99
        16                 75
        17                 75
        18                 72
        19                 30
        20                 11
        21                 38
        22                 26
        23                 21
1       0                  20
        1                  31
        2                  23
        3                   5
        4                  12
        5                   6
     